In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import math
from sklearn.preprocessing import normalize
import warnings
from datetime import timedelta
warnings.filterwarnings("ignore")

In [2]:
import os
owd = "F:\Projects\DataProcessor"
print("owd: ", owd)

owd:  F:\Projects\DataProcessor


In [ ]:
os.chdir(owd)
data_dir = owd+"\StudentLife Data"
os.chdir(data_dir)
dir_list = [x for x in os.listdir('.') if "student" in x]

# Skipping Student 0, because of bad data.
dir_list = dir_list[1:]
# dir_list = dir_list[1:3]
print("DirList that will be precessed: ", dir_list)
converter_dict = {'time':pd.to_datetime}

for folder in dir_list:
    print("Working on: ", folder)
    
    path = data_dir + "\\"+ folder
    os.chdir(path) 
    file_list = [file[:-12]  for file in os.listdir() if "train_x" in file]
    feature_test_list = []
    feature_train_list = []
    
    # select student only if all the feature present.
    
    if (len(file_list) < 11):
        continue
        
    
    for file in file_list:
        
        raw_feature_train = pd.read_csv(file+ "_train_x.csv",
                                          skip_blank_lines=False,
                                          index_col=0,
                                          converters=converter_dict)

        raw_feature_train_y_indices = pd.read_csv(file+ "_train_y_indices.csv", 
                                                  skip_blank_lines=False, 
                                                  index_col=0)

        # to bring the values from 0-4.
        raw_feature_train["stress_level"] += -1

        # Hardcore indexing, to convert single index to multi so that max, min and avg can be taken easily.
        # Train set.
        list_a = []
        list_b = raw_feature_train.index.values

        indices = list(raw_feature_train_y_indices.indices.values)
        indices = [-1] + indices

        for indx in range(1, len(indices)) :
            list_a += [indices[indx] for i in range(indices[indx]-indices[indx-1])]

        index_keys = [
            np.array(list_a),
            np.array(list_b)        
        ]

        raw_feature_train.set_index(keys=index_keys, inplace=True)
        
        feature_list = list(raw_feature_train.columns.values) 
        feature_list.remove("student_id")
        feature_list.remove("time")
        feature_list.remove("stress_level")
        
        # Colapsing Multindex to fin min, max and mean of the seq. 
        raw_feature_train_min = raw_feature_train.min(level=0)
        raw_feature_train_max = raw_feature_train.max(level=0)
        

        # Mean, Median and Count also creating the column list.
        raw_feature_train_mean = raw_feature_train.mean(level=0)
        raw_feature_train_median = raw_feature_train.median(level=0)
        raw_feature_train_sum = raw_feature_train.sum(level=0)
        raw_feature_train_count = raw_feature_train.count(level=0)
        
        # Explicit delta given to the model. Since all the counts are same, we just need to pick any one column.
        raw_feature_train_count = raw_feature_train_count.iloc[:,0]

        #getting time delta in secconds
        time_delta = raw_feature_train_max["time"] - raw_feature_train_min["time"] 
        time_delta = time_delta.apply(timedelta.total_seconds)
        
        
        raw_feature_train = pd.concat([  raw_feature_train_min.loc[:,feature_list], 
                                         raw_feature_train_max.loc[:, feature_list],
                                         raw_feature_train_mean.loc[:, feature_list],
                                         raw_feature_train_median.loc[:, feature_list],
                                         raw_feature_train_sum.loc[:, feature_list],
                                         time_delta,
                                         raw_feature_train_count], 
                                         axis=1,
                                         ignore_index=True)
        
        # preparing column list and renaming the columns.
        column_list = [file + "_"+feature + "_min" for feature in feature_list]
        column_list = column_list + [file + "_"+feature + "_max" for feature in feature_list]
        column_list = column_list + [file + "_"+feature + "_mean" for feature in feature_list]
        column_list = column_list + [file + "_"+feature + "_median" for feature in feature_list]
        column_list = column_list + [file + "_"+feature + "_sum" for feature in feature_list]
        column_list = column_list + [file + "_time_delta"]
        column_list = column_list + [file + "_count"]
        raw_feature_train.columns = column_list
        
        # Reseeting Index
        raw_feature_train.reset_index(drop=True, inplace=True)

        feature_train_list.append(raw_feature_train)
    
    
    #splitting stress labels. Need to do this only once, since stress level will be the same for all features.
    stress_levels = raw_feature_train_min["stress_level"]
    stress_levels.reset_index(drop=True, inplace=True)
    
    # grabbing the student_id.
    student_id = raw_feature_train_min["student_id"]
    student_id.reset_index(drop=True, inplace=True)
    
    # concatenating every thing for final dataset.
    concatenated_train = pd.concat([student_id]+feature_train_list+[stress_levels], axis=1)
    
    var_exists = 'train_set' in locals() or 'train_set' in globals()
    
    if var_exists:
        train_set = train_set.append(other=concatenated_train, ignore_index=True)
    else:
        train_set = pd.DataFrame(concatenated_train)

os.chdir(data_dir+'/VarableLength Aggregates')
train_set.to_csv("variable_interval_aggregate_train.csv", index=True)


DirList that will be precessed:  ['student 1', 'student 10', 'student 12', 'student 13', 'student 14', 'student 15', 'student 16', 'student 17', 'student 18', 'student 19', 'student 2', 'student 20', 'student 22', 'student 23', 'student 24', 'student 25', 'student 27', 'student 3', 'student 30', 'student 31', 'student 32', 'student 33', 'student 34', 'student 35', 'student 36', 'student 39', 'student 4', 'student 41', 'student 42', 'student 43', 'student 44', 'student 45', 'student 46', 'student 47', 'student 49', 'student 5', 'student 50', 'student 51', 'student 52', 'student 53', 'student 54', 'student 56', 'student 57', 'student 58', 'student 59', 'student 7', 'student 8', 'student 9']
Working on:  student 1
Working on:  student 10
